In [3]:
import pyodbc
import pandas as pd

In [181]:
drivers = pd.read_csv("../data/etl_out/VehicleCrashFact.csv")
roaddim = pd.read_csv("../data/etl_out/RoadDim.csv")
vehicles = pd.read_csv("../data/etl_out/VehicleDim.csv")
location = pd.read_csv("../data/etl_out/LocationAreaDim.csv")
weather = pd.read_csv("../data/etl_out/WeatherFact.csv")
datehour = pd.read_csv("../data/etl_out/DateHourDim.csv")

In [146]:
names = ['drivers', 'roaddim', 'vehicles', 'location', 'weather', 'datehour']
tables = [drivers, roaddim, vehicles, location, weather, datehour]
for name, table in zip(names, tables):
    rows_with_na = table[table.isna().any(axis=1)]
    print(name, len(rows_with_na))

drivers 0
roaddim 0
vehicles 0
location 0
weather 0
datehour 0


In [8]:
conn.close()

In [4]:
DRIVER_NAME = 'SQL SERVER'
SERVER_NAME = 'WAW-2AT9QIDXNUV'
DATABASE_NAME = 'vehicle_crashes_dwh_clone'

connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={{{SERVER_NAME}}};
    DATABASE={{{DATABASE_NAME}}};
    Trust_Connection=yes;
    uid=tymek;
    pwd=password;
"""

In [5]:
try:
    conn = pyodbc.connect(connection_string)
    print('Connection succesful:', conn)
except Exception as e:
    print(e)

Connection succesful: <pyodbc.Connection object at 0x0000021AB5204D30>


In [6]:
def test_connection(conn):
    cursor = conn.cursor()
    query = "SELECT @@SERVERNAME"
    cursor.execute(query)
    rows = cursor.fetchall()
    return rows[0]

In [7]:
test_connection(conn)

('WAW-2AT9QIDXNUV', )

In [152]:
# conn.close()

### RoadDim

In [96]:
insert_road_dim = """
BEGIN TRY
    INSERT INTO RoadDim (RoadName, RouteType, RoadKey) 
    VALUES (?, ?, ?);
END TRY
BEGIN CATCH
    IF ERROR_NUMBER() = 2601 OR ERROR_NUMBER() = 2627  -- Duplicate key violation error numbers
    BEGIN
        PRINT 'Duplicate key, skipping row'
    END
    ELSE
    BEGIN
        THROW;
    END
END CATCH
"""

cursor = conn.cursor()

for index, row in roaddim.iterrows():
    try:
        cursor.execute(insert_road_dim, row['RoadName'], row['RouteType'], row['RoadKey'])
    except Exception as e:
        print("An error occurred:", e)
        print(row)
        break

conn.commit()
cursor.close()

### VehicleDim

In [98]:
insert_vehicle_dim = """
BEGIN TRY
    INSERT INTO VehicleDim (Make, Year, BaseModel, VehicleKey, BodyClass, Cylinders, Displacement, Transmission, Drivetrain, FuelType, CityMPG, HighwayMPG) 
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
END TRY
BEGIN CATCH
    IF ERROR_NUMBER() = 2601 OR ERROR_NUMBER() = 2627  -- Duplicate key violation error numbers
    BEGIN
        PRINT 'Duplicate key, skipping row'
    END
    ELSE
    BEGIN
        THROW;
    END
END CATCH
"""

cursor = conn.cursor()

for index, row in vehicles.iterrows():
    try:
        cursor.execute(insert_vehicle_dim, 
                       row['Make'], 
                       row['Year'], 
                       row['BaseModel'], 
                       row['VehicleKey'], 
                       row['BodyClass'], 
                       row['Cylinders'], 
                       row['Displacement'], 
                       row['Transmission'], 
                       row['Drivetrain'], 
                       row['FuelType'], 
                       row['CityMPG'], 
                       row['HighwayMPG'])
    except Exception as e:
        print("An error occurred:", e)
        print(row)
        break

conn.commit()
cursor.close()

### DateHourDim

In [100]:
datehour.columns

Index(['Datetime', 'DateHourKey', 'Hour', 'TimeOfDay', 'DayNumber',
       'WeekDayNumber', 'WeekDayName', 'WeekendFlag', 'MonthNumber',
       'MonthName', 'Year', 'HolidayFlag', 'HolidayName'],
      dtype='object')

In [153]:
insert_time_dim = """
BEGIN TRY
    INSERT INTO DateHourDim (Datetime, DateHourKey, Hour, TimeOfDay, DayNumber, WeekDayNumber, WeekDayName, WeekendFlag, MonthNumber, MonthName, Year, HolidayFlag, HolidayName) 
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
END TRY
BEGIN CATCH
    IF ERROR_NUMBER() = 2601 OR ERROR_NUMBER() = 2627  -- Duplicate key violation error numbers
    BEGIN
        PRINT 'Duplicate key, skipping row'
    END
    ELSE
    BEGIN
        THROW;
    END
END CATCH
"""

cursor = conn.cursor()

for index, row in datehour.iterrows():
    try:
        cursor.execute(insert_time_dim, 
                       row['Datetime'], 
                       row['DateHourKey'], 
                       row['Hour'], 
                       row['TimeOfDay'], 
                       row['DayNumber'], 
                       row['WeekDayNumber'], 
                       row['WeekDayName'], 
                       row['WeekendFlag'], 
                       row['MonthNumber'], 
                       row['MonthName'], 
                       row['Year'], 
                       row['HolidayFlag'], 
                       row['HolidayName'])
    except Exception as e:
        print("An error occurred:", e)
        print(row)
        break

conn.commit()
cursor.close()

### LocationAreaDim

In [106]:
insert_location_dim = """
BEGIN TRY
    INSERT INTO LocationAreaDim (LocationAreaKey, Zipcode, MailCity, ShapeLength, ShapeArea, CentroidLatitude, CentroidLongitude) 
    VALUES (?, ?, ?, ?, ?, ?, ?);
END TRY
BEGIN CATCH
    IF ERROR_NUMBER() = 2601 OR ERROR_NUMBER() = 2627  -- Duplicate key violation error numbers
    BEGIN
        PRINT 'Duplicate key, skipping row'
    END
    ELSE
    BEGIN
        THROW;
    END
END CATCH
"""

cursor = conn.cursor()

for index, row in location.iterrows():
    try:
        cursor.execute(insert_location_dim, 
                       row['LocationAreaKey'], 
                       row['Zipcode'], 
                       row['MailCity'], 
                       row['ShapeLength'], 
                       row['ShapeArea'], 
                       row['CentroidLatitude'], 
                       row['CentroidLongitude'])
    except Exception as e:
        print("An error occurred:", e)
        print(row)
        break

conn.commit()
cursor.close()

### WeatherFact

In [121]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [182]:
weather['WeatherKey'] = weather['WeatherKey'].astype(str)
weather['LocationAreaKey'] = weather['LocationAreaKey'].astype(str)
weather['DateHourKey'] = weather['DateHourKey'].astype(str)

In [185]:
insert_weather_fact = """
BEGIN TRY
    INSERT INTO WeatherFact (Temperature, Humidity, Precipitation, Rain, Snow, WindSpeed, WindDirection, LocationAreaKey, DateHourKey, WeatherKey) 
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
END TRY
BEGIN CATCH
    IF ERROR_NUMBER() = 2601 OR ERROR_NUMBER() = 2627  -- Duplicate key violation error numbers
    BEGIN
        PRINT 'Duplicate key, skipping row'
    END
    ELSE
    BEGIN
        THROW;
    END
END CATCH
"""

# insert_weather_fact = """
#     INSERT INTO WeatherFact (Temperature, Humidity, Precipitation, Rain, Snow, WindSpeed, WindDirection, LocationAreaKey, DateHourKey, WeatherKey) 
#     VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
# """

cursor = conn.cursor()

for index, row in weather.iterrows():
    try:
        cursor.execute(insert_weather_fact, 
                       row['Temperature'], 
                       row['Humidity'], 
                       row['Precipitation'], 
                       row['Rain'], 
                       row['Snow'], 
                       row['WindSpeed'], 
                       row['WindDirection'], 
                       row['LocationAreaKey'], 
                       row['DateHourKey'], 
                       row['WeatherKey'])
    except Exception as e:
        print("An error occurred:", e)
        print(row)
        break

conn.commit()
cursor.close()


In [186]:
drivers.columns

Index(['ReportNumber', 'VehicleCrashKey', 'DriverAtFault',
       'DriverInjurySeverity', 'DriverSubstanceAbuse', 'DriverDistractedBy',
       'VehicleType', 'VehicleMovement', 'VehicleGoingDir',
       'VehicleDamageExtent', 'SpeedLimit', 'ParkedVehicle',
       'SubstanceAbuseContributed', 'VehiclesCrashedTotal', 'VehicleKey',
       'LocalCaseNumber', 'AgencyName', 'ACRSReportType', 'HitRun',
       'LaneDirection', 'LaneNumber', 'NumberOfLanes', 'RoadGrade',
       'NonTraffic', 'OffRoadIncident', 'AccidentAtFault', 'CollisionType',
       'SurfaceCondition', 'Light', 'TrafficControl', 'Junction',
       'IntersectionType', 'RoadAlignment', 'RoadCondition', 'RoadDivision',
       'Latitude', 'Longitude', 'RoadKey', 'CrossStreetKey',
       'NonMotoristTotal', 'NonMotoristInjury', 'NonMotoristFatal',
       'DateHourKey', 'LocationAreaKey'],
      dtype='object')

In [191]:
insert_query = """
INSERT INTO VehicleCrashFact (
    ReportNumber, VehicleCrashKey, DriverAtFault, DriverInjurySeverity, DriverSubstanceAbuse,
    DriverDistractedBy, VehicleType, VehicleMovement, VehicleGoingDir, VehicleDamageExtent,
    SpeedLimit, ParkedVehicle, SubstanceAbuseContributed, VehiclesCrashedTotal, VehicleKey,
    LocalCaseNumber, AgencyName, ACRSReportType, HitRun, LaneDirection, LaneNumber,
    NumberOfLanes, RoadGrade, NonTraffic, OffRoadIncident, AccidentAtFault, CollisionType,
    SurfaceCondition, Light, TrafficControl, Junction, IntersectionType, RoadAlignment,
    RoadCondition, RoadDivision, Latitude, Longitude, RoadKey, CrossStreetKey,
    NonMotoristTotal, NonMotoristInjury, NonMotoristFatal, DateHourKey, LocationAreaKey
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""


cursor = conn.cursor()

# Insert each row from the DataFrame into the database table
for index, row in drivers.iterrows():
    try:
        cursor.execute(insert_query, 
                       row['ReportNumber'],
                       row['VehicleCrashKey'],
                       row['DriverAtFault'],
                       row['DriverInjurySeverity'],
                       row['DriverSubstanceAbuse'],
                       row['DriverDistractedBy'],
                       row['VehicleType'],
                       row['VehicleMovement'],
                       row['VehicleGoingDir'],
                       row['VehicleDamageExtent'],
                       row['SpeedLimit'],
                       row['ParkedVehicle'],
                       row['SubstanceAbuseContributed'],
                       row['VehiclesCrashedTotal'],
                       row['VehicleKey'],
                       row['LocalCaseNumber'],
                       row['AgencyName'],
                       row['ACRSReportType'],
                       row['HitRun'],
                       row['LaneDirection'],
                       row['LaneNumber'],
                       row['NumberOfLanes'],
                       row['RoadGrade'],
                       row['NonTraffic'],
                       row['OffRoadIncident'],
                       row['AccidentAtFault'],
                       row['CollisionType'],
                       row['SurfaceCondition'],
                       row['Light'],
                       row['TrafficControl'],
                       row['Junction'],
                       row['IntersectionType'],
                       row['RoadAlignment'],
                       row['RoadCondition'],
                       row['RoadDivision'],
                       row['Latitude'],
                       row['Longitude'],
                       row['RoadKey'],
                       row['CrossStreetKey'],
                       row['NonMotoristTotal'],
                       row['NonMotoristInjury'],
                       row['NonMotoristFatal'],
                       row['DateHourKey'],
                       row['LocationAreaKey'])    
    except Exception as e:
        print("An error occurred:", e)
        print(row)
        break

# Commit the transaction
conn.commit()

# Close the connection
cursor.close()